## A very basic illustration of hash-based commit/reveal

In [1]:
!pip install pycryptodome

In [2]:
# Note: "Crypto" here is pycryptodome
#from Crypto import Random
from Crypto.Random import random
#from Crypto.Util import number
from Crypto.Hash import keccak  # keccak256 is the Ethereum utility hash func

Assume Bob and Alice would like to play a round of "rock, paper, scissors"

Since they are in separate locations, they can't just "count it off" on their hands...

But if they just send their choices to one another it is simply too tempting for one of them to wait for the other's move to arrive before sending, and therefore to cheat.

So they are going to make use of "hash commitment" to keep things honest.


In [198]:

rps_choices = ['rock', 'paper', 'scissors']

def rps_winner( choiceA, choiceB):
    if choiceA == choiceB:
        return 'tie'
    return choiceA if rps_choices.index(choiceA) == (rps_choices.index(choiceB)+1)%3 else choiceB   
    

In [267]:
# Pck a winner if you know the pther choice
def cheat( myChoice, otherChoice):
    if rps_winner(myChoice, otherChoice) != myChoice:  
        cheatChoice =  rps_choices[(rps_choices.index(otherChoice)+1)%3]
        print(f'Cheat by saying {cheatChoice}')
        return cheatChoice
    else:
        print(f'No reason to cheat')        
        return None

In [230]:
def salted_hash(choice, salt):
    # both arg are stings
    keccak_hash = keccak.new(digest_bits=256)
    s = choice + salt
    return keccak_hash.update(bytes(s,'ascii')).hexdigest()
    

In [231]:
def get_salt(bits=256):
    return str(random.getrandbits(bits))

They have decided that, after they select their respective moves, Alice will send a hash value representing her choice to Bob.

When Bob receives it, he can't tell what choice it represents, but he knows that it does uniquely represent a choice, and that when Alice does send her actual value he will be able to prove for himself that it was her choice all along.

So as soon as he receives her commitment, he feels safe to send her his choice in the clear.

When Alice gets it, she feels safe to reveal hers to Bob. Send sends the choice to him, along with a "salt" value she used to make the hash unique.

Bob verifies that when he computes the hash of Alice's revealed value with the salt, it does in fact result in her committed hash.

And now they can see who won.


"A_" at the beginning of a variable name means that the value is private to Alice.

`B_foo = A_foo` means that Alice has sent `foo` to Bob

In [281]:
# bob and alice both choose
A_aliceChoice = rps_choices[random.randint(0,2)]
B_bobChoice = rps_choices[random.randint(0,2)]

print(f"Alice's choice: {A_aliceChoice}\nBob's choice: {B_bobChoice}")

Alice's choice: paper
Bob's choice: rock


In [282]:
# alice creates a commitment hash
A_aliceSalt = get_salt()
A_aliceCommit = salted_hash(A_aliceChoice, A_aliceSalt)

print( f'Salt: {A_aliceSalt}\nCommitment hash: {A_aliceCommit}')

Salt: 8468474995766385997617628695752620376159245890350207812849077563936068443602
Commitment hash: ac290d8d0cfdd4ee147457d4bbd518cd9042fb3112b47bc03a0910167af4873b


In [283]:
# Alice sends Bob her commitment
B_aliceCommit = A_aliceCommit

In [284]:
# Now that Bob has Alice's committment, sends his actual choice
A_bobChoice = B_bobChoice

In [285]:
# maybe Alice tries to cheat by changing her choice

# A_aliceChoice = cheat(A_aliceChoice, A_bobChoice) or A_aliceChoice


In [286]:
# Alice sends reveal to bob
B_aliceReveal = ( A_aliceChoice, A_aliceSalt) # choice + salt

In [287]:
# Alice finds the winner:

a_winningChoice = rps_winner(A_aliceChoice, A_bobChoice)
      
results  = 'Tie' if a_winningChoice == 'tie' else 'Alice won' if a_winningChoice == A_aliceChoice else 'Bob won'

print(f'Alice says: {results}')


Alice says: Alice won


In [288]:
# Bob verifies Alice's commitment 

(B_aliceChoice, B_aliceSalt) = B_aliceReveal

B_aliceCheated = B_aliceCommit != salted_hash(B_aliceChoice, B_aliceSalt)
    
if B_aliceCheated:
    print("Alice has cheated!")
else:
    print("Alice was honest")       

Alice was honest


In [289]:
# finds the winner

b_winningChoice = rps_winner(B_aliceChoice, B_bobChoice)
    
b_results  = 'Tie' if b_winningChoice == 'tie' else 'Alice won' if b_winningChoice == B_aliceChoice else 'Bob won'

print(f'Bob says: {"CHEATING! " if B_aliceCheated else ""}{results}')

if b_winningChoice != a_winningChoice:
    print("Something has gone terribly wrong")  




Bob says: Alice won
